# Abell 1656: the Coma Cluster of Galaxies
This notebook is based on the EURO-VO tutorial: "Abell 1656: The Coma Cluster of Galaxies" (http://www.euro-vo.org/?q=science/scientific-tutorials) and shows how to perform steps 1 to 8 from within a Jupyter notebook. Steps 9 to 11 are not implemented. 

## Introduction
The goals of this notebook tutorial are:
 - Examine the Coma cluster of galaxies (Abell 1656) using services and data from the virtual observatory withinin a jupyter notebook in order to perform a quick evaluation of the mean redshift and velocity dispersion of the cluster. Both measurements are important to study the evolution of galaxy clusters. 
 - Use redshifts and photometry (Petrosian r magnitude) of the SDSS survey and then add redshifts of the CAIRNS survey (Rines et al. 2003) in order to improve the completeness of the redshift sample.
 
For our analysis we need to following python packages:

In [ ]:
import ipyaladin.aladin_widget as ipyal
from astroquery.vizier import Vizier
from astroquery.xmatch import XMatch
from astropy.table import Table, vstack
from astropy import units as u
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
print('All done and ready to go :) ')

## Display the region of Abell 1656 in Aladin lite
We start by displaying the Coma Cluster in an Aladin Lite widget. We will centre the DSS2 colour images (`survey='P/DSS2/color'`) on Abell 1656 (`target='A1656'`) and set the field of view to 0.7deg (`fov=0.7`). At the distance of Coma, our field of view corresponds to approximately 1Mpc a region large enough for our purposes. 

Note that if you are using Jupyter lab you can now open a second python3 notebook, click on the "Python 3" button in the top, right corner of this notebook to switch kernel. A new window will pop up and you can then select this notebook (i.e. "Abel1656...") as kernel. This will link the two notebooks such that they see the same variables ect. You may use the second linked notebook to be able to look at a Aladin Lite widget the entire time, while doing the analysis work in this notebook. Please see the ipyaladin webpage for more information on how to use the widget in Jupyter lab. 

In [ ]:
aladin = ipyal.Aladin(target='A1656', fov=0.7, survey='P/DSS2/color')

In [ ]:
aladin

As with any Aladin Lite implementation, you can interact with this widget: 
 - to zoom in and out place you mouse pointer on top of the image and scroll. 
 - with <img src="Images/ipyaladin_layer.png" alt="the Layer Button" style="width:30px; display: inline-block;"/>  you can select other image surveys and manage the current view.
 - if you like to look at another target, you can use the search field <img src="Images/ipyaladin_search.png" alt="the Search Button" style="width:30px; display: inline-block;"/> to get there. 

These interactions can also be steered by changing properties of the variable `aladin`. If for example, after zooming in and out, you wanted to set the FoV again to 0.7deg, do:

In [ ]:
aladin.fov = 0.7

## Load the SDSS-DR9 catalog and select galaxies
In this section, we will search the SDSS DR9 catalogue at the VizieR catalogue service and then download all entries that are located within 40arcmin of the centre of the A1656. We start by querying VizieR for all catalogues that match the search term `SDSS DR9`: 

In [ ]:
catalog_list_sdss = Vizier.find_catalogs('SDSS DR9')
for k, v in catalog_list_sdss.items():
    print(k, ': ', v.description)

We are interested in data from the main SDSS DR9 photometric catalogues: "The SDSS Photometric Catalog, Release 9 (Adelman-McCarthy+, 2012)". For an intial exploration of the content of this catalogue we do a query around 10arcmin of the centre of the Coma Cluster. We then print the output of the query, which is a list of Astropy Tables. 

In [ ]:
results_test_sdss = Vizier.query_region("A1656", radius="0d10m0s", catalog='V/139')
print(results_test_sdss)

As you can see there is one table with 50 entries in our resulting list of tables, let's see which columns it contains:

In [ ]:
print(results_test_sdss[0].colnames)

For our science target, we will only need the coordinates (`RA_ICRS` and `DE_ICRS`), r-band magnitudes (`rmag`) and redshifts (`zsp`) of the galaxies in the Coma cluster. Hence, we restrict our query such that we only get these columns. SDSS furthermore provides information on the type of source (galaxies: `cl` = 3) and observing mode (primary sources: `mode` = 1). To only get galaxies, which are also prinarmy sources, we implement a filtering of the columns. The way to implement these restrictions, is to create a specialiced instance of the Vizier class, which we will call v1.

In [ ]:
v1 = Vizier(columns=['RA_ICRS', 'DE_ICRS', 'rmag', 'zsp'],
            column_filters={'cl': '==3', 'mode': '==1'})

Above you might have noticed that the number of sources in the queried region is surprisingly small and round. This is because by default VizieR will only return 50 entries. If we want to get all sources within the queried region, we have to increase the `ROW_LIMIT`:

In [ ]:
v1.ROW_LIMIT = -1

Now we that we have prepared everything, we can query VizieR for all galaxies, which are primary sources and within 40arcmin of the centre of the Coma cluster. Note that VizieR know that `A1656` is the centre of the Coma cluster. If you wanted to search at some other place in the sky, you could also give coordinates (in the form of Astropy's `SkyCoord`) instead of `'A1656'`.

In [ ]:
results_coma_sdss = v1.query_region('A1656', radius='0d40m0s', catalog='V/139')
print(results_coma_sdss)

As you can see, the result of our query includes data from one catalogue for 23771 objects (i.e. the table has 23771 rows). The output from this query is again a list of Astropy Table objects, so we assign the table to the variable `sdss_dr9` and work with this from here on. 

In [ ]:
sdss_dr9 = results_coma_sdss[0]
sdss_dr9

## Identify the brightest sources as being stars contaminating the sample
We have already restricted our sample to sources that are classified as galaxies (`cl` = 3). However, for very bright sources, stars might be confused for galaxies. To test this and exclude any contamination from stars, we now take a closer look at the brightests sources. To do so we select sources brighter than 11.5 mag in r-band (`rmag < 11.5`) and check with the Aladin lite widget what these sources look like. 

In [ ]:
stars = sdss_dr9[sdss_dr9['rmag'] < 11.5]
aladin.add_table(stars)
print('Our sample contains {} really bright sources.'.format(len(stars)))

With the line `aladin.add_table(stars)`, we have added symbols to the Aladin Lite widget at the location of the brightest sources (i.e. stars). If you now scroll back up to the Aladin Lite widget and zoom out, you will be able to find all the brightest sources. By looking at each source (zoom in on them), you will find that these are indeed stars. 

## Build a subset of galaxies with photometry and redshift in SDSS
Now on to exploring the galaxies in our sample. 
We again build a subset, this time for all sources fainter than 11.5mag (to leave out the stars identified in the section above) but brighter than 17.77mag, which is the completeness limit the SDSS spectroscopic sample. 

In [ ]:
zsp17 = sdss_dr9[(sdss_dr9['rmag'] > 11.5) & (sdss_dr9['rmag'] < 17.77)]
zsp17

## Improve the completeness with other sources of redshifts in Vizier
As you can see in the table above, not all galaxies in our zsp17 sample have redshift measurements (some rows have '--' in the 'zsp' column, i.e. they are masked). So in order to improve the completeness of our sample we will now use Vizier to search for redshifts in the Rines et al (2003) catalogue. First find all catalogues that match the search terms 'redshifts Rines 2003':

In [ ]:
catalog_list_rines = Vizier.find_catalogs('redshifts Rines 2003')
for k, v in catalog_list_rines.items():
    print(k, ': ', v.description)

Of these two catalogues, we are interested in the 'J/AJ/126/2152' catalogue. So again, let's take a quick look at this catalogue. 

In [ ]:
results_test_rines = Vizier.query_region("A1656", radius="0d10m0s", catalog='J/AJ/126/2152')
print(results_test_rines)

In [ ]:
results_test_rines[0]

In [ ]:
results_test_rines[1]

After inspecting the result of the test query, it becomes clear that we are in particular interested in the 'galaxies' table of the catalogue. We aim to find redshifts in this table for galaxies, which have no SDSS redshifts. In a first step we divide the zsp17 sample into two subsamples: the ones that already have a redshift measurement - zsp17_with - and the ones without redshift measurements - zsp17_wihtout (the entry in the table is a NaN). Then we use the CDS XMatch service (http://cdsxmatch.u-strasbg.fr/) via the `astroquery.XMatch.query` module to look for spatial crossmatches of galaxies from the zsp17_wihtout sample with entries the 'J/AJ/126/2152/galaxies' table hosted at Vizier. 

In [ ]:
ind = np.isnan(zsp17['zsp']) 
zsp17_with = zsp17[~ind]
zsp17_without = zsp17[ind]
zsp17_without.write('zsp17_without.csv', overwrite=True)
xmatch_sdss_rines = XMatch.query(cat1=open('zsp17_without.csv'), 
                                 cat2='vizier:J/AJ/126/2152/galaxies',
                                 max_distance=5 * u.arcsec, colRA1='RA_ICRS',
                                 colDec1='DE_ICRS')
xmatch_sdss_rines

## Build the final catalogue including the Rines et al. (2003) redshifts
The resulting table of the cross-match above contains 27 rows, so we have found recession velocity ('cz') measurements for 27 galaxies. Now let's add these data to the zsp17_with table.

In [ ]:
zsp17_with['cz'] = zsp17_with['zsp'] * 300000
zsp17_without_new = xmatch_sdss_rines[zsp17_with.colnames]
zsp17_final = vstack([zsp17_with, zsp17_without_new])
zsp17_final

Now we have a table with all galaxies that have redshift measurements from SDSS and all additional galaxies, for which redshift measurements were obtained by Rines et al. (2003). Overall, the sample of galaxies within 40arcmin of the centre of the Coma cluster and with redshift measurements now contains 513 galaxies. Before we get started on the analysis of the data, we can have a look at the sources in the sample by loading the table into the Aladin Lite widget. They will appear in different colours than the stars before. 

In [ ]:
aladin.add_table(zsp17_final)

## Determine the cz distribution, the cluster average velocity and velocity dispersion
Based on the 513 galaxies, we can now analyse the recession velocity and velocity dispersion of the Abell 1656 galaxy cluster. First we visualise the recession velocity distribution of the entire sample:

In [ ]:
sns.set_style("darkgrid")
fig = plt.figure(figsize=(7.5, 6.0))
ax = fig.add_axes([0.17, 0.17, 0.77, 0.77])
sns.distplot(zsp17_final['cz'], rug=True, ax=ax, kde=False)

Note how there is a large range of recession velocities in our sample. We are only interested in the range of recession velocities of the Coma cluster. These are around the large peak at low velocities. To focus on these, we restrict ourselves to recession velocities between 3000 and 11000 km/s. For easy analysis we first create the subsample `zsp17_coma` and then visualise the distribution of the recession velocity in this subsample:

In [ ]:
ind = (zsp17_final['cz'] > 3000.) & (zsp17_final['cz'] < 11000.)
zsp17_coma = zsp17_final[ind]

sns.set_style("darkgrid")
fig = plt.figure(figsize=(7.5, 6.0))
ax = fig.add_axes([0.17, 0.17, 0.77, 0.77])
sns.distplot(zsp17_coma['cz'], rug=True, ax=ax, kde=False)
ax.set_xlim([3000, 11000])

These are data of all galaxies for which we collected redshift/recession velocity measurements and which are in the vincinity of the cluster (both spatially and in recession velocity). Now let's calculate the mean recession velocity of the cluster and its velocity dispersion:

In [ ]:
print('The mean velocity in Coma is: {0:.1f}km/s and \
the standard deviation (i.e. velocity dispersion):\
{1:.1f}km/s'.format(np.mean(zsp17_coma['cz']), np.std(zsp17_coma['cz'])))

This is in agreement with more refined analyses (e.g. Sohn et al. 2017, ApJS, 229, 20). When looking back at the results of the query for the Rines et al. (2003) catalogue, you can also see that in their `clusters` table they gave `cz` = 6973km/s and `sigmap_3s_` = 1042km/s for the Coma cluster, again in good agreement with our result. 

In [ ]:
results_test_rines[0]